In [1]:
from data_cleaning import DataCleaning
from data_extraction import DataExtractor
from database_utils import DatabaseConnector

import pandas as pd
import numpy as np

In [2]:
db_connector = DatabaseConnector()
db_extractor = DataExtractor()
cleaner = DataCleaning()
credentials = db_connector.read_db_creds('db_creds.yaml')
engine = db_connector.init_db_engine(credentials)

In [3]:
with engine.execution_options(isolation_level='AUTOCOMMIT').connect() as conn:
    # Milestone 2 Step 4
    db_connector.list_db_tables(engine)
    
    df = db_extractor.read_rds_table(conn, 'legacy_users')
    

['legacy_store_details', 'legacy_users', 'orders_table']


In [4]:
df = cleaner.clean_user_data(df=df)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   first_name     15284 non-null  object        
 1   last_name      15284 non-null  object        
 2   date_of_birth  15284 non-null  datetime64[ns]
 3   company        15284 non-null  object        
 4   email_address  15284 non-null  object        
 5   address        15284 non-null  object        
 6   country        15284 non-null  category      
 7   country_code   15284 non-null  category      
 8   phone_number   15284 non-null  object        
 9   join_date      15284 non-null  datetime64[ns]
 10  user_uuid      15284 non-null  object        
dtypes: category(2), datetime64[ns](2), object(7)
memory usage: 1.4+ MB


In [6]:
df

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
index,,,,,,,,,,,
0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579
...,...,...,...,...,...,...,...,...,...,...,...
15315,Marta,Rogge,1981-03-03,Dehmel,baererklothilde@trubin.com,Ziegertstr. 60\n93330 Stollberg,Germany,DE,(05917) 549662,2000-03-29,8a77629e-7ca1-409f-b22c-c24056bd4eb1
15316,Erna,Hoffmann,1967-10-28,Atzler Seifert AG & Co. KGaA,dunjafischer@vollbrecht.de,Henkallee 186\n33456 Sankt Goarshausen,Germany,DE,+49(0)7384 51073,2018-03-13,5f57209e-8695-4863-b9e7-084a4ba02808
15317,Konstantinos,Thanel,1954-08-05,Fritsch Ehlert GmbH,rpruschke@gotthard.com,Steffi-Rose-Platz 16\n12365 Apolda,Germany,DE,(07856) 050049,2007-07-21,6f16e0ce-9b07-4479-a151-7efdd35408aa
